### Import

In [ ]:
import tensorflow as tf
!pip install tensorflow-gan
import tensorflow_gan as tfgan
from tensorflow.keras import layers, models, optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install sdv
from sdv.evaluation.single_table import get_column_plot
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import evaluate_quality
from sdv.evaluation.single_table import run_diagnostic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.5 MB/s eta 0:00:00


# Codice WGAN

In [ ]:
# link & mount to google drive to import data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
d_DDoS = pd.read_csv('/content/drive/My Drive/Colab Notebooks/progettoTesi/Dataset2017/improved/Pre-processed_versions/d_DDoS35.csv')
dfDDoS = d_DDoS.drop(['L_BENIGN','L_Infiltration_Portscan','L_DDoS','L_DoS','L_BruteForce','L_Botnet'], axis=1)
dfDDoS.shape

(95133, 35)

In [ ]:
X_train = dfDDoS.copy()

# Architettura WGAN

In [ ]:
noise_dim = 100  #@param      #Aggiustare in base alle performance
lr=0.0001 #@param
lrG=0.0001 #@param
lrC=0.0001 #@param
epochs = 1000 #@param
batch_size = 64 #@param

In [ ]:
def build_generator(noise_dim):
        model = tf.keras.Sequential(name="Generator_model")
        model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=noise_dim.shape))
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dense(512, activation='relu'))
        model.add(tf.keras.layers.Dense(35, activation='sigmoid'))  #35 feature - Sigmoid for [0, 1] range output data
        return model

def build_critic():
     model = tf.keras.Sequential(name="Discriminator_model")
     model.add(tf.keras.layers.Dense(512, activation='relu', input_dim=35))
     model.add(tf.keras.layers.Dense(256, activation='relu'))
     model.add(tf.keras.layers.Dense(128, activation='relu'))
     model.add(tf.keras.layers.Dense(1, activation=None))
     return model

## Creazione GAN e training

In [ ]:
# Build the WGAN Model
gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=build_generator,
    discriminator_fn=build_critic,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    params={'batch_size': batch_size, 'noise_dim': noise_dim},
    generator_optimizer=tf.keras.optimizers.legacy.Adam(lrG, 0.5),
    discriminator_optimizer=tf.keras.optimizers.legacy.Adam(lrC, 0.5)
)

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.




```
# For a dataset with 95,133 samples and 35 features, a common approach is to
 choose a batch size that is a power of 2 (e.g., 32, 64, 128) to leverage GPU
  parallelism efficiently. However, you may need to experiment with different
   batch sizes to find the one that balances computational efficiency and
   model convergence.
```



In [ ]:
# Replace 'FILE_ID' with the actual file ID from your Google Drive shareable link
file_id = '1-0k5rDdZPwN6jKdYmVprLLINZBRSzLsM'
#https://drive.google.com/file/d/1-0k5rDdZPwN6jKdYmVprLLINZBRSzLsM/view?usp=drive_link
# Construct the download link
download_link = f'https://drive.google.com/uc?id={file_id}'


In [ ]:
# CIT - https://www.tensorflow.org/guide/estimator#create_input_functions

def train_input_fn():
  #ds = tf.data.Dataset.from_tensor_slices(dict(X_train))
  attack_file = tf.keras.utils.get_file('d_DDoS35.csv', origin=download_link)
  attack = tf.data.experimental.make_csv_dataset(attack_file, batch_size=64)
  attack_batches = (attack.cache().repeat().shuffle(500).prefetch(tf.data.AUTOTUNE))
  return attack_batches

In [ ]:
steps_per_eval = 500 #@param
max_train_steps = 8000 #@param
batches_for_eval_metrics = 100 #@param

In [ ]:
# Disable noisy output.
tf.autograph.set_verbosity(0, False)

import time

cur_step = 0
start_time = time.time()



# --------------------------- TRAIN SECTION ------------------------------
while cur_step < max_train_steps:
  next_step = min(cur_step + steps_per_eval, max_train_steps)

  start = time.time()
  gan_estimator.train(train_input_fn, max_steps=next_step) #!!!!!!!!!!!

  steps_taken = next_step - cur_step
  time_taken = time.time() - start
  print('Time since start: %.2f min' % ((time.time() - start_time) / 60.0))
  print('Trained from step %i to %i in %.2f steps / sec' % (cur_step, next_step, steps_taken / time_taken))
  cur_step = next_step

  # Calculate some metrics.
  metrics = gan_estimator.evaluate(train_input_fn, steps=batches_for_eval_metrics)
  print(metrics)



Instructions for updating:
Use tf.keras instead.


66359469/66359469 [==============================] - 0s 0us/step


TypeError: ignored

In [ ]:
result = gan_estimator.evaluate(train_input_fn, steps=10)

#### Plottino metriche valutazione

In [ ]:
# Plot the metrics vs step.
plt.title('MNIST Frechet distance per step')
plt.plot(steps, frechet_distances)
plt.figure()
plt.title('MNIST Score per step')
plt.plot(steps, mnist_scores)
plt.plot(steps, real_mnist_scores)
plt.show()

# Generazione dati dopo il training

In [ ]:
generated_data = gan_estimator.predict(train_input_fn)



generated_df.head()

### Valutazione dei dati generati

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(dfDDoS)

quality_report = evaluate_quality(
    real_data=dfDDoS,
    synthetic_data=gen_ddos_samples,
    metadata=metadata)

In [ ]:
diagnostic_report = run_diagnostic(
    real_data=dfDDoS,
    synthetic_data=gen_ddos_samples,
    metadata=metadata)